
# **Fine Tuning DeBerta Model for Fake news Detection**

#**Etapes du projet:**

- Load Pretrained Model
- Preparer le Tokenizer
- Prétraiter et Tokeniser les données
- Preparer les arguments de Training
- Initialiser le Trainer
- Calculer l’accuracy
- Preparer l’UI

In [1]:
#Installer les bibliotheques à utiliser: bitsandbytes, peft, datasets, accelerate
!pip install bitsandbytes
!pip install peft
!pip install datasets
!pip install accelerate
import accelerate
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import DistilBertTokenizer, DistilBertModel, Trainer, TrainingArguments, AutoModelForSequenceClassification, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
import re

def get_device_map() -> str:
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device_map()  # 'cpu'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.2 MB/s eta 0:00:00
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


ImportError: cannot import name 'research' from partially initialized module 'bitsandbytes' (most likely due to a circular import) (/usr/local/lib/python3.10/dist-packages/bitsandbytes/__init__.py)

# **Loading the Base Model to Fine-tune**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Load pre-trained model
## LLaMa2
"""model = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    cache_dir="/data/yash/base_models",
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          cache_dir="/data/yash/base_models"
                                         )"""

## deBerta
model_name = "microsoft/deberta-v3-base"  # You can use other variants like "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Adjust num_labels to your task

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **Processing Dataset**

In [ ]:
data = load_dataset("ErfanMoosaviMonazzah/fake-news-detection-dataset-English", cache_dir = "/data/datasets")


Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8267 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label'],
        num_rows: 30000
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label'],
        num_rows: 8267
    })
})

In [ ]:
def process_data(example):
  example ['prompt']="the news title is: " + example['title']+"\n\n"+"The subject of the news is: " + example['subject']+"\n\n" + "The body text of the news is: " + example['text'] + '\n\n'
  example['complete_prompt'] = example['prompt']+ "this news is " + ("fake" if example['label']==0 else "real")
  return example


In [ ]:
data = data.map(process_data)


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8267 [00:00<?, ? examples/s]

In [ ]:
data


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label', 'prompt', 'complete_prompt'],
        num_rows: 30000
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label', 'prompt', 'complete_prompt'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label', 'prompt', 'complete_prompt'],
        num_rows: 8267
    })
})

In [ ]:
print(data['train']['complete_prompt'][2])

the news title is: Federal Reserve governor Powell's policy views, in his own words

The subject of the news is: politicsNews

The body text of the news is: President Donald Trump on Thursday tapped Federal Reserve Governor Jerome Powell to become head of the U.S. central bank, promoting a soft-spoken centrist to replace Janet Yellen when her term expires in February 2018. In five years as a Fed Governor Jerome Powell has been a consistent, middle of the road voice, backing the consensus crafted by Fed chair Janet Yellen that interest rates should be raised slowly so labor markets could recover, that financial stability risks were muted, and that new regulations had made the economy safer. Following is a collection of quotes from select policy speeches he has delivered since 2015: On Rates:  “The financial crisis did significant damage to the productive capacity of our economy, and the damage was of a character, extent, and duration that cannot be fully known today…It seems plausible t

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def tokenize_dataset(example):
    response = tokenizer(example['complete_prompt'], padding='max_length', truncation=True, max_length=512)
    return response


In [ ]:
data = data.map(tokenize_dataset, batched=True)


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8267 [00:00<?, ? examples/s]

In [ ]:
data = data.remove_columns(['title', 'subject', 'text', 'Unnamed: 0'])
data.set_format('torch', columns=['input_ids','prompt', 'attention_mask', 'label','complete_prompt'])


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['date', 'label', 'prompt', 'complete_prompt', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30000
    })
    validation: Dataset({
        features: ['date', 'label', 'prompt', 'complete_prompt', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['date', 'label', 'prompt', 'complete_prompt', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8267
    })
})

# **Configuring model for fine-tuning**

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [ ]:
print_trainable_parameters(model)

trainable params: 184423682 || all params: 184423682 || trainable%: 100.0


In [ ]:
"""for param in model.parameters():
  param.requires_grad = False"""

'for param in model.parameters():\n  param.requires_grad = False'

# **Starting the Training Loop**

In [ ]:

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    save_steps=1000,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
)

# Fine-tune the entire model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.model.save_pretrained('outputs')


# Loading the fine-tuned model

In [ ]:
"""output_dir = './outputs'
import transformers
# Reload model in FP16 and merge it with LoRA weights
base_model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    cache_dir="/data/base_models/",
    #device_map='auto',
)

model = PeftModel.from_pretrained(base_model, output_dir)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", cache_dir="/data/base_models/")"""

# Testing model output of fine-tuned model

In [ ]:
%%writefile app.py
# Streamlit app interface
st.title("Fake News Detection with DeBERTa")

uploaded_text = st.text_area("Enter news text here...")

if st.button("Classify"):
    if uploaded_text:
        inputs = tokenizer(uploaded_text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1).item()
        class_name = "Fake" if predicted_class == 0 else "Real"
        st.write(f"Predicted class: {class_name}")
    else:
        st.write("Please enter some text to classify.")

# Function to classify text
def classify_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    return predicted_class

In [ ]:
!apt-get install nodejs npm
!npm install -g localtunnel

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501